In [1]:
import requests                          # For sending HTTP requests to the Bible website
import pandas as pd                      # For handling structured tabular data
import time                              # For adding delay between API requests
from bs4 import BeautifulSoup            # For parsing HTML content from the Bible website

# Step 1: Define Bible books and number of chapters
books = {
    "gal": 6,
    "eph": 6,
    "phi": 4,
    "col": 4,
    "1 the": 5,
    "2 the": 3,
    "1 tim": 6
}

# Bible version mapping (Bible.com version IDs)
version = {
    "GNBDC": "416",
}

# Define the scraping function
def scrape_bible_csv(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    # Loop through versions, books, and chapters
    for version_name, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    # Construct the URL
                    url = f"{base_url}{version_id}/{book.lower()}.{chapter}"
                    print(f"Scraping: {url} ({version_name})")

                    # Fetch page content
                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version_name}). Skipping...")
                        continue

                    # Parse HTML
                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version_name}). Skipping...")
                        continue

                    # Extract verse number and text, fallback to index if number is missing
                    for idx, verse in enumerate(verses, start=1):
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_text_tag = verse.find('span', class_='ChapterContent_content__RrUqA')

                        verse_num = verse_num_tag.text.strip() if verse_num_tag else str(idx)
                        verse_text = verse_text_tag.text.strip() if verse_text_tag else ""

                        if verse_text:
                            all_verses.append({
                                "Book": book.upper(),
                                "Chapter": chapter,
                                "Verse": verse_num,
                                "Text": verse_text,
                                "Version": version_name
                            })

                    # Sleep to avoid overwhelming the server
                    time.sleep(1)

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version_name}): {e}. Skipping...")

    # Save collected verses to CSV
    df = pd.DataFrame(all_verses)
    df.to_csv('GNBDC.csv', index=False, encoding='utf-8-sig')
    print("✅ GNBDC Bible verses have been saved to 'GNBDC.csv'.")

# Run the scraping function
scrape_bible_csv(books, version)

Scraping: https://www.bible.com/bible/416/gal.1 (GNBDC)
Scraping: https://www.bible.com/bible/416/gal.2 (GNBDC)
Scraping: https://www.bible.com/bible/416/gal.3 (GNBDC)
Scraping: https://www.bible.com/bible/416/gal.4 (GNBDC)
Scraping: https://www.bible.com/bible/416/gal.5 (GNBDC)
Scraping: https://www.bible.com/bible/416/gal.6 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.1 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.2 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.3 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.4 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.5 (GNBDC)
Scraping: https://www.bible.com/bible/416/eph.6 (GNBDC)
Scraping: https://www.bible.com/bible/416/phi.1 (GNBDC)
No verses found for phi 1 (GNBDC). Skipping...
Scraping: https://www.bible.com/bible/416/phi.2 (GNBDC)
No verses found for phi 2 (GNBDC). Skipping...
Scraping: https://www.bible.com/bible/416/phi.3 (GNBDC)
No verses found for phi 3 (GNBDC). Skipping...
Scraping: https://w